In [1]:
import pandas as pd
import numpy as np

In [2]:
# import data
data = pd.read_csv('hcp_drug_prediction_data_uncleaned.csv')
print(data.shape)

(1204, 75)


In [3]:
# brief glimps of the data
# print(data.head)
print(data.columns) # sub_use = drug label; 0 = no; 1 = yes

Index(['Subject', 'Gender', 'Age', 'PSQI_Score', 'PicSeq_AgeAdj',
       'CardSort_AgeAdj', 'Flanker_AgeAdj', 'PMAT24_A_CR', 'PMAT24_A_SI',
       'PMAT24_A_RTCR', 'ReadEng_AgeAdj', 'PicVocab_AgeAdj',
       'ProcSpeed_AgeAdj', 'DDisc_SV_1mo_200', 'DDisc_SV_6mo_200',
       'DDisc_SV_1yr_200', 'DDisc_SV_3yr_200', 'DDisc_SV_5yr_200',
       'DDisc_SV_10yr_200', 'DDisc_SV_1mo_40K', 'DDisc_SV_6mo_40K',
       'DDisc_SV_1yr_40K', 'DDisc_SV_3yr_40K', 'DDisc_SV_5yr_40K',
       'DDisc_SV_10yr_40K', 'DDisc_AUC_200', 'DDisc_AUC_40K', 'VSPLOT_TC',
       'VSPLOT_CRTE', 'VSPLOT_OFF', 'SCPT_TP', 'SCPT_TN', 'SCPT_FP', 'SCPT_FN',
       'SCPT_TPRT', 'SCPT_SEN', 'SCPT_SPEC', 'SCPT_LRNR', 'IWRD_TOT',
       'IWRD_RTC', 'ListSort_AgeAdj', 'CogFluidComp_AgeAdj',
       'CogEarlyComp_AgeAdj', 'CogTotalComp_AgeAdj', 'CogCrystalComp_AgeAdj',
       'ER40_CR', 'ER40_CRT', 'ER40ANG', 'ER40FEAR', 'ER40HAP', 'ER40NOE',
       'ER40SAD', 'AngAffect_Unadj', 'AngHostil_Unadj', 'AngAggr_Unadj',
       'FearAffect

In [4]:
# gender counts
data.groupby('Gender').size()

Gender
F    655
M    549
dtype: int64

In [5]:
# sub_use counts
data.groupby('sub_use').size()

sub_use
0    911
1    293
dtype: int64

In [6]:
# gender x sub_use counts
data.groupby(['Gender', 'sub_use']).size()

Gender  sub_use
F       0          554
        1          101
M       0          357
        1          192
dtype: int64

In [7]:
data.groupby('Age').size()

Age
22-25    247
26-30    526
31-35    417
36+       14
dtype: int64

In [8]:
# get NA counts in columns
# missing values = NA
na_counts_cols = data.isnull().sum().sort_values(ascending = False)
print(na_counts_cols)

CogTotalComp_AgeAdj      19
CogFluidComp_AgeAdj      18
CogEarlyComp_AgeAdj      15
SCPT_TPRT                12
CogCrystalComp_AgeAdj    10
                         ..
ReadEng_AgeAdj            0
PicVocab_AgeAdj           0
ProcSpeed_AgeAdj          0
ListSort_AgeAdj           0
Subject                   0
Length: 75, dtype: int64


In [9]:
# get NA counts in rows
na_counts_rows = data.isnull().sum(axis = 1).sort_values(ascending = False)
print(na_counts_rows)
sum(na_counts_rows > 0) # number of subjects missing ANY measure

96     42
54     42
87     42
921    42
248    42
       ..
797     0
798     0
799     0
800     0
0       0
Length: 1204, dtype: int64


35

In [10]:
# figure out which and how many subjects are missing a measure 
missing_data_rows = (na_counts_rows > 0) # figure out which subjects are missing a measure
data.loc[missing_data_rows].groupby(['Gender','sub_use']).size() # number of subjects (gender x sub_use) with missing data

Gender  sub_use
F       0          22
        1           4
M       0           6
        1           3
dtype: int64

In [11]:
# remove subjects with a missing measure from the data
clean_data = data[-missing_data_rows]
print(clean_data.shape)
clean_data.groupby(['Gender', 'sub_use']).size()

(1169, 75)


/Applications/anaconda3/envs/psych_pheno/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Gender  sub_use
F       0          532
        1           97
M       0          351
        1          189
dtype: int64

In [12]:
clean_data['Age'].unique()

array(['22-25', '26-30', '31-35', '36+'], dtype=object)

In [13]:
age_recoded = np.where(clean_data['Age']=='22-25', 1, 
                  np.where(clean_data['Age']=='26-30', 2, 
                      np.where(clean_data['Age']=='31-35', 3, 4))) 

In [14]:
clean_data = pd.concat([clean_data, pd.DataFrame({'Age_recoded': age_recoded})], axis = 1)
clean_data.drop('Age', axis = 1)

,Subject,Gender,PSQI_Score,PicSeq_AgeAdj,CardSort_AgeAdj,Flanker_AgeAdj,PMAT24_A_CR,PMAT24_A_SI,PMAT24_A_RTCR,ReadEng_AgeAdj,...,InstruSupp_Unadj,PercStress_Unadj,SelfEff_Unadj,NEOFAC_A,NEOFAC_O,NEOFAC_C,NEOFAC_N,NEOFAC_E,sub_use,Age_recoded
0,100004.0,M,8.0,112.44,88.64,111.01,19.0,0.0,15590.0,96.91405,...,34.8,55.4,39.6,27.0,15.0,28.0,24.0,25.0,0.0,1.0
1,100206.0,M,6.0,118.78,104.94,116.55,20.0,0.0,18574.5,103.44410,...,46.0,57.8,44.0,29.0,23.0,26.0,21.0,32.0,0.0,2.0
2,100307.0,F,4.0,103.45,109.92,101.90,17.0,2.0,11839.0,98.73000,...,52.3,57.9,37.5,37.0,24.0,35.0,15.0,37.0,0.0,2.0
3,100408.0,M,4.0,125.19,100.77,113.51,7.0,12.0,3042.0,125.64000,...,51.1,46.8,56.4,33.0,29.0,34.0,15.0,33.0,1.0,3.0
4,100610.0,M,4.0,101.69,115.18,114.18,23.0,0.0,12280.0,132.41240,...,48.4,37.8,53.2,31.0,33.0,36.0,7.0,15.0,0.0,2.0
5,101006.0,F,2.0,70.00,94.30,92.33,11.0,8.0,6569.0,101.16970,...,43.3,48.8,38.8,30.0,23.0,39.0,13.0,29.0,1.0,3.0
6,101107.0,M,9.0,97.37,105.69,96.19,14.0,5.0,6880.0,112.97570,...,32.6,51.9,68.4,28.0,35.0,37.0,12.0,28.0,0.0,1.0
7,101208.0,F,6.0,95.64,82.44,84.46,8.0,11.0,6805.0,89.81669,...,51.3,43.9,55.7,32.0,37.0,34.0,15.0,28.0,0.0,3.0
8,101309.0,M,5.0,93.36,86.03,103.37,15.0,4.0,12558.0,92.68638,...,62.9,42.9,62.4,36.0,22.0,36.0,2.0,26.0,0.0,2.0
9,101410.0,M,8.0,111.31,106.84,100.58,19.0,0.0,21640.0,101.29010,...,62.9,46.0,49.5,32.0,26.0,35.0,12.0,29.0,0.0,2.0


In [15]:
# output the clean data
clean_data.to_csv('hcp_drug_prediction_data_cleaned.csv', index = False)